In [128]:
import random
import time 
from colorsys import hsv_to_rgb
import board
from digitalio import DigitalInOut, Direction
from PIL import Image, ImageDraw, ImageFont
from adafruit_rgb_display import st7789
import numpy as np


In [129]:
h, s, v = 0.5, 1.0, 1.0  # 예시로 사용될 HSV 값
r, g, b = hsv_to_rgb(h, s, v)  # HSV를 RGB로 변환
print(f"RGB: ({r}, {g}, {b})")

RGB: (0.0, 1.0, 1.0)


In [130]:



# Joystick 클래스: 조이스틱 입력 처리
class Joystick:
    def __init__(self):
        self.cs_pin = DigitalInOut(board.CE0)
        self.dc_pin = DigitalInOut(board.D25)
        self.reset_pin = DigitalInOut(board.D24)
        self.BAUDRATE = 24000000

        self.spi = board.SPI()
        self.disp = st7789.ST7789(
            self.spi,
            height=240,
            y_offset=80,
            rotation=180,
            cs=self.cs_pin,
            dc=self.dc_pin,
            rst=self.reset_pin,
            baudrate=self.BAUDRATE,
        )

        self.button_U = DigitalInOut(board.D17)
        self.button_U.direction = Direction.INPUT

        self.button_D = DigitalInOut(board.D22)
        self.button_D.direction = Direction.INPUT

        self.button_L = DigitalInOut(board.D27)
        self.button_L.direction = Direction.INPUT

        self.button_R = DigitalInOut(board.D23)
        self.button_R.direction = Direction.INPUT

        self.button_A = DigitalInOut(board.D5)
        self.button_A.direction = Direction.INPUT

        self.button_B = DigitalInOut(board.D6)
        self.button_B.direction = Direction.INPUT

        self.width = self.disp.width
        self.height = self.disp.height


# 점수 그리기 함수
def draw_score(draw_obj, score, font):
    draw_obj.text((10, 10), f"Score: {score}", font=font, fill=(0, 0, 0))


# 게임 시작 메뉴 그리기 함수
def draw_main_menu(screen, canvas, joystick, font):
    splash_image = Image.open('/home/kau-jeon/TA-ESW/3.Game/image/PyCar.png').resize((joystick.width, joystick.height))
    screen.paste(splash_image, (0, 0))  # 'canvas' 대신 'screen'을 사용하여 이미지 붙이기
    canvas.text((40, 80), "PyCar : Racing Car Game", font=font, fill=(0, 255, 135))  # 검은색 텍스트
    canvas.text((40, 120), "Press Any Button to Start", font=font, fill=(255, 255, 0))  # 빨간색 텍스트


# 게임 종료 메시지 표시 함수
def draw_game_over(draw_obj, font_large, font_small, score):
    draw_obj.text((40, 80), "Game Over", font=font_large, fill=(255, 0, 0))
    draw_obj.text((40, 120), f"Final Score: {score}", font=font_small, fill=(255, 255, 255))
    draw_obj.text((40, 160), "Press Any Button to Restart", font=font_small, fill=(255, 255, 255))




In [131]:


# Car 클래스: 플레이어 차량
class Car:
    def __init__(self, width, height, image_path):
        self.image = Image.open(image_path).resize((width, height))
        self.width = width
        self.height = height
        self.position = np.array([120 - self.width / 2, 240 - self.height - 10,
                                  120 + self.width / 2, 240 - 10])

    def move(self, command):
        if not command['move']:
            return  # 움직임 없음

        if command['up_pressed']:
            self.position[1] -= 20
            self.position[3] -= 20
        if command['down_pressed']:
            self.position[1] += 20
            self.position[3] += 20
        if command['left_pressed']:
            self.position[0] -= 20
            self.position[2] -= 20
        if command['right_pressed']:
            self.position[0] += 20
            self.position[2] += 20

    def draw(self, screen):
    # 이미지를 그대로 원본 크기로 표시
        screen.paste(self.image, (int(self.position[0]), int(self.position[1])), self.image)

    def check_out_of_screen(self, screen_width, screen_height=240):
        # 화면 경계를 벗어나지 않도록 위치 제한
        if self.position[0] < 0:
            self.position[0] = 0
            self.position[2] = self.width
        elif self.position[2] > screen_width:
            self.position[0] = screen_width - self.width
            self.position[2] = screen_width

        if self.position[1] < 0:
            self.position[1] = 0
            self.position[3] = self.height
        elif self.position[3] > screen_height:
            self.position[1] = screen_height - self.height
            self.position[3] = screen_height

    def check_crash(self, obstacle):
        if (self.position[0] < obstacle.x + obstacle.width and
            self.position[2] > obstacle.x and
            self.position[1] < obstacle.y + obstacle.height and
            self.position[3] > obstacle.y):
            return True
        return False


In [132]:
class ObstacleCar:
    # 장애물 이미지 경로 목록
    obstacle_images = [
        '/home/kau-jeon/TA-ESW/3.Game/image/RacingCar16.png',
        '/home/kau-jeon/TA-ESW/3.Game/image/RacingCar11.png',
        '/home/kau-jeon/TA-ESW/3.Game/image/RacingCar18.png',
        '/home/kau-jeon/TA-ESW/3.Game/image/RacingCar19.png',
        '/home/kau-jeon/TA-ESW/3.Game/image/RacingCar20.png',
    ]

    def __init__(self, width, height, all_obstacles=None):
        """
        초기화 메서드
        :param width: 장애물 너비
        :param height: 장애물 높이
        :param all_obstacles: 다른 장애물 목록 (충돌 검사에 사용)
        """
        self.width = width
        self.height = height
        self.image = Image.open(random.choice(self.obstacle_images)).resize((self.width, self.height)).convert("RGBA")
        self.all_obstacles = all_obstacles if all_obstacles is not None else []
        self.reset_position()

    def reset_position(self):
        """
        장애물 위치와 이동 속도 초기화
        충돌 없는 위치가 선택될 때까지 반복
        """
        while True:
            self.x = random.randint(0, 240 - self.width)  # 화면 내 x 좌표
            self.y = random.randint(-100, 0)  # 화면 위쪽 y 좌표
            self.dx = random.choice([-1, 1]) * random.randint(2, 5)  # 좌우 이동 속도
            self.dy = random.randint(3, 6)  # 아래로 내려오는 속도
            if not self.is_colliding_with_other_obstacles():
                break

    def is_colliding_with_other_obstacles(self):
        """
        다른 장애물과 충돌 여부 확인
        """
        for obstacle in self.all_obstacles:
            if (self.x < obstacle.x + obstacle.width and
                self.x + self.width > obstacle.x and
                self.y < obstacle.y + obstacle.height and
                self.y + self.height > obstacle.y):
                return True
        return False

    def move(self):
        """
        장애물 이동
        화면을 벗어나면 위치를 재설정
        """
        self.x += self.dx
        self.y += self.dy
        if self.y > 240 or self.x < -self.width or self.x > 240:
            self.reset_position()
            return True
        return False

    def draw(self, screen):
        """
        화면에 장애물을 그리기
        :param screen: 디스플레이 화면
        """
        screen.paste(self.image, (int(self.x), int(self.y)), self.image)

    def check_crash(self, car):
        """
        플레이어 차량과 충돌 여부 확인
        :param car: 플레이어 차량 객체
        :return: 충돌 여부 (True/False)
        """
        return (
            self.x < car.position[0] + car.width and
            self.x + self.width > car.position[0] and
            self.y < car.position[1] + car.height and
            self.y + self.height > car.position[1]
        )


In [ ]:


class TriggerObstacle:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.image = Image.open('/home/kau-jeon/TA-ESW/3.Game/image/trigger.png').resize((self.width, self.height)).convert("RGBA")
        self.reset_position()



    def draw(self, screen):
        screen.paste(self.image, (self.x, self.y), self.image)

    def check_crash(self, car):
        if (self.x < car.position[0] + car.width and
            self.x + self.width > car.position[0] and
            self.y < car.position[1] + car.height and
            self.y + self.height > car.position[1]):
            return True
        return False


    def reset_position(self):
        self.x = int(random.randint(0, 240 - self.width))  # 정수 변환
        self.y = int(random.randint(-100, 0))             # 정수 변환
        self.dy = random.randint(15, 20)  # 속도는 float도 가능

    def move(self):
        self.y += self.dy
        if self.y > 240:
            self.reset_position()
            return True
        return False


def draw_level(canvas, score, font_small):
    """레벨 표시 함수"""
    if score <= 70:
        level = 1
    elif 70 < score <= 150:
        level = 2
    elif 150 < score <= 300:
        level = 3    
    else:
        level = 4

    canvas.text((190, 10), f"Level: {level}", fill=(0, 0, 0), font=font_small)

        

def game():
    joystick = Joystick()  # 조이스틱 객체 생성

    # 디스플레이 화면 설정
    screen = Image.new("RGB", (joystick.width, joystick.height))
    canvas = ImageDraw.Draw(screen)
    font_large = ImageFont.load_default()  # 큰 폰트
    font_small = ImageFont.load_default()  # 작은 폰트

    # 차선 설정
    lane_width = 10
    lane_height = 40
    lane_margin = 20
    lane_x = (joystick.width - lane_width) / 2
    lanes = [[lane_x, -i * (lane_height + lane_margin)] for i in range(10)]

    score = 0
    game_on = False
    game_over = False

    while True:
        command = {'move': False, 'up_pressed': False, 'down_pressed': False, 'left_pressed': False, 'right_pressed': False}

        # 게임 시작 메뉴
        if not game_on and not game_over:
            canvas.rectangle((0, 0, joystick.width, joystick.height), fill=(169, 169, 169))  # 회색 배경
            draw_main_menu(screen, canvas, joystick, font_large)
            joystick.disp.image(screen)

            # 버튼 입력 대기
            if not joystick.button_A.value or not joystick.button_B.value:
                game_on = True
                score = 0
                obstacles = [ObstacleCar(35,60) for _ in range(3)]  # 장애물 초기화
                trigger_obstacles = [TriggerObstacle(60, 55)]  # 트리거 장애물 초기화
                my_car = Car(35, 60, "/home/kau-jeon/TA-ESW/3.Game/image/RacingCar17.png")  # 차량 초기화
                time.sleep(1)  # 게임 시작 대기

        elif game_over:
            # 게임 오버 화면
            canvas.rectangle((0, 0, joystick.width, joystick.height), fill=(0, 0, 0))  # 검은 배경
            draw_game_over(canvas, font_large, font_small, score)  # 게임 오버 메시지와 점수 표시
            joystick.disp.image(screen)

            # 버튼 입력 대기
            if not joystick.button_A.value or not joystick.button_B.value:
                game_over = False  # 게임 오버 상태 종료
                game_on = False  # 게임 초기 상태로 복귀
                time.sleep(1)  # 버튼 입력 대기

        else:
            # 게임 진행 중
            canvas.rectangle((0, 0, joystick.width, joystick.height), fill=(169, 169, 169))  # 회색 배경

            # 차선 그리기
            for lane in lanes:
                lane[1] += 5  # 차선 이동
                if lane[1] > joystick.height:
                    lane[1] = -lane_height - lane_margin
                canvas.rectangle([lane[0], lane[1], lane[0] + lane_width, lane[1] + lane_height], fill=(255, 255, 255))

            # 차량 이동
            command['move'] = True
            command['up_pressed'] = not joystick.button_U.value
            command['down_pressed'] = not joystick.button_D.value
            command['left_pressed'] = not joystick.button_L.value
            command['right_pressed'] = not joystick.button_R.value
            my_car.move(command)
            my_car.check_out_of_screen(joystick.width)

            if score <= 70:  # Level 1
                # 장애물 이동 및 그리기
                for trigger_obstacle in trigger_obstacles:
                    if trigger_obstacle.move():
                        score += 10
                    trigger_obstacle.draw(screen)

                # 충돌 검사
                    if trigger_obstacle.check_crash(my_car):
                        game_over = True  # 게임 오버 상태 설정
                        break
            elif 70 < score <= 150:  # Level 2
                # 장애물 수 증가
                if len(trigger_obstacles) < 2:
                    trigger_obstacles.append(TriggerObstacle(60, 55))

                for trigger_obstacle in trigger_obstacles:
                    if trigger_obstacle.move():  # 속도 증가
                        score += 10
                    trigger_obstacle.draw(screen)

                    # 충돌 검사
                    if trigger_obstacle.check_crash(my_car):
                        game_over = True
                        break
            elif 150 < score <= 300:  # Level 3
                # Level 2에서는 기존 장애물에 속도와 빈도를 증가
                for obstacle in obstacles:
                    obstacle.dy += 0.01 # 속도 증가
                    if obstacle.move():
                        score += 10
                    obstacle.draw(screen)

                    # 충돌 검사
                    if obstacle.check_crash(my_car):
                        game_over = True  # 게임 오버 상태 설정
                        break

            elif 300 < score <= 450:
    # Level 3에서는 추가적인 장애물을 생성
                if len(obstacles) < 4:
                    obstacles.append(ObstacleCar(random.randint(30, 40), 50))
                for obstacle in obstacles:
                    obstacle.dy += 0.01  # 속도 더 큰 증가
                    if obstacle.move():
                        score += 20  # 점수 보너스
                    obstacle.draw(screen)

                    # 충돌 검사
                    if obstacle.check_crash(my_car):
                        game_over = True  # 게임 오버 상태 설정
                        break
            elif score > 300:  # 게임 종료 조건
                # 게임 종료 화면 표시
                endgame_image = Image.open("/home/kau-jeon/TA-ESW/3.Game/image/endgame.png")
                endgame_image = endgame_image.resize((joystick.width, joystick.height))  # 화면 크기에 맞게 이미지 조정
                screen.paste(endgame_image, (0, 0))  # 이미지를 화면에 표시
                joystick.disp.image(screen)

                time.sleep(3)  # 3초 동안 종료 화면 표시
                game_on = False  # 게임 종료 상태 설정
                break  # 게임 루프 종료



            # 플레이어 차량 그리기
            my_car.draw(screen)

            # 레벨 표시
            draw_level(canvas, score, font_small)

            # 점수 그리기
            draw_score(canvas, score, font_small)

            # 화면에 표시
            joystick.disp.image(screen)
            time.sleep(0.03)  # 게임 속도 조정

game()  
